<a href="https://colab.research.google.com/github/ishapugalia/Abstractive-Text-Summarization/blob/main/model_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import os
import pandas as pd
import re

In [3]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [4]:
import tensorflow as tf

In [5]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Dataset csv/train.csv")


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
d_articles=pd.DataFrame(data.iloc[:10000,[0,1]])
print(d_articles[:1])
d_highlights=pd.DataFrame(data.iloc[:10000,[0,2]])
print(d_highlights.columns)
#print(d_articles)

                                         id  \
0  0001d1afc246a7964130f43ae940af6bc6c57f01   

                                             article  
0  By . Associated Press . PUBLISHED: . 14:11 EST...  
Index(['id', 'highlights'], dtype='object')


In [8]:
def cleantext(text):
    text = str(text)
    text=text.split()
    words=[]
    for t in text:
        if t.isalpha():
            words.append(t)
    text=" ".join(words)
    text=text.lower()
    text=re.sub(r"what's","what is ",text)
    text=re.sub(r"it's","it is ",text)
    text=re.sub(r"\'ve"," have ",text)
    text=re.sub(r"i'm","i am ",text)
    text=re.sub(r"\'re"," are ",text)
    text=re.sub(r"n't"," not ",text)
    text=re.sub(r"\'d"," would ",text)
    text=re.sub(r"\'s","s",text)
    text=re.sub(r"\'ll"," will ",text)
    text=re.sub(r"can't"," cannot ",text)
    text=re.sub(r" e g "," eg ",text)
    text=re.sub(r"e-mail","email",text)
    text=re.sub(r"9\\/11"," 911 ",text)
    text=re.sub(r" u.s"," american ",text)
    text=re.sub(r" u.n"," united nations ",text)
    text=re.sub(r"\n"," ",text)
    text=re.sub(r":"," ",text)
    text=re.sub(r"-"," ",text)
    text=re.sub(r"\_"," ",text)
    text=re.sub(r"\d+"," ",text)
    text=re.sub(r"[$#@%&*!~?%{}()]"," ",text)
    
    return text

In [9]:
d_articles["article"]= d_articles["article"].apply(lambda x: cleantext(x))

In [10]:
d_highlights["highlights"]= d_highlights["highlights"].apply(lambda x: cleantext(x))

In [11]:
VOCAB_SIZE = 29999
tokenizer = Tokenizer(num_words=VOCAB_SIZE)
tokenizer.fit_on_texts(list(d_articles.article))
article_sequences = tokenizer.texts_to_sequences(list(d_articles.article))
art_word_index = tokenizer.word_index
len(art_word_index)

75952

In [12]:
art_vocab = {}
counter = 0
for word in art_word_index.keys():
    if art_word_index[word] == 0:
        print("found 0!")
        break
    if art_word_index[word] > VOCAB_SIZE:
        continue
    else:
        art_vocab[word] = art_word_index[word]
        counter += 1

In [13]:
tokenizer.fit_on_texts(d_highlights)
summary_sequences = tokenizer.texts_to_sequences(d_highlights.highlights)
sum_word_index = tokenizer.word_index
len(sum_word_index)

75952

In [14]:
sum_vocab = {}
counter = 0
for word in sum_word_index.keys():
    if sum_word_index[word] == 0:
        print("found 0!")
        break
    if sum_word_index[word] > VOCAB_SIZE:
        continue
    else:
        sum_vocab[word] = sum_word_index[word]
        counter += 1

In [15]:
MAX_LEN = 400
pad_article = pad_sequences(article_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

In [16]:
MAX_LEN_S= 100
pad_highlight = pad_sequences(summary_sequences, maxlen=MAX_LEN_S, padding='post', truncating='post')

In [17]:
pad_article.shape
pad_highlight

array([[ 4164,   286,     5, ...,     0,     0,     0],
       [  772,  6955,   152, ...,     0,     0,     0],
       [ 2725,    31,  3608, ...,     0,     0,     0],
       ...,
       [ 7177,     7,   961, ...,     0,     0,     0],
       [ 1370,    11,  2122, ...,     0,     0,     0],
       [23613, 21278,     7, ...,     0,     0,     0]], dtype=int32)

In [18]:
embeddings_index = {}
with open('/content/drive/MyDrive/Colab Notebooks/glove.6B.200d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

In [18]:
def embedding_matrix_creater(embedding_dimention, word_index):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dimention))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [19]:
art_embedding_matrix = embedding_matrix_creater(200, word_index=art_vocab)
art_embedding_matrix.shape

(30000, 200)

In [ ]:
df=pd.DataFrame(art_embedding_matrix)
df.to_excel("/content/drive/MyDrive/Colab Notebooks/trained/art_embedding_matrix.xlsx")


In [ ]:
import pickle

In [ ]:
filehandler = open("/content/drive/MyDrive/Colab Notebooks/trained/art_vocab", 'wb')
pickle.dump(art_vocab, filehandler)

In [ ]:
filehandler = open("/content/drive/MyDrive/Colab Notebooks/trained/sum_vocab", 'wb')
pickle.dump(sum_vocab, filehandler)

In [ ]:
#file = open('/content/drive/MyDrive/Colab Notebooks/trained/art_vocab.txt', 'rb')

# dump information to that file
#data1 = pickle.load(file)
#print(data1)

In [20]:
sum_embedding_matrix = embedding_matrix_creater(200, word_index=sum_vocab)
sum_embedding_matrix.shape

(30000, 200)

In [ ]:
art_embedding_matrix[10]

In [21]:
def embedding_layer_creater(VOCAB_SIZE, EMBEDDING_DIM, MAX_LEN, embedding_matrix):
  embedding_layer = tf.keras.layers.Embedding(input_dim = VOCAB_SIZE, 
                                    output_dim = EMBEDDING_DIM,
                                    input_length = MAX_LEN,
                                    weights = [embedding_matrix],
                                    trainable = False)
  
  return embedding_layer



In [22]:
encoder_embedding_layer = embedding_layer_creater(30000,200, MAX_LEN, art_embedding_matrix)

In [23]:
decoder_embedding_layer = embedding_layer_creater(30000,200, MAX_LEN_S, sum_embedding_matrix)

In [ ]:
if not os.path.exists("/content/drive/MyDrive/Colab Notebooks/trained"):
    os.makedirs("/content/drive/MyDrive/Colab Notebooks/trained")

In [ ]:
!pip install chart-studio

In [24]:
from numpy.random import seed
seed(1)

from sklearn.model_selection import train_test_split
import logging

import chart_studio.plotly as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import pandas as pd
import pydot


import keras
from keras import backend as k
k.set_learning_phase(1)
from keras.preprocessing.text import Tokenizer
from keras import initializers
from tensorflow.keras.optimizers import RMSprop
from keras.models import Sequential,Model
from keras.layers import Dense,LSTM,Dropout,Input,Activation,Add,concatenate, Embedding, RepeatVector
from keras.layers.advanced_activations import LeakyReLU,PReLU
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from tensorflow.keras.optimizers import Adam

/usr/local/lib/python3.7/dist-packages/keras/backend.py:450: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


In [25]:
from keras.layers import TimeDistributed
import gensim

In [26]:
# Hyperparams

MAX_LEN = 400
MAX_LEN_S=100
VOCAB_SIZE=30000
EMBEDDING_DIM = 200
HIDDEN_UNITS = 256
#VOCAB_SIZE = VOCAB_SIZE + 1

LEARNING_RATE = 0.002
BATCH_SIZE = 32
EPOCHS = 10

In [27]:
from keras import backend as K 
import gensim
from numpy import *
import numpy as np
import pandas as pd 
import re
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
import tensorflow as tf

In [28]:
encoder_inputs = Input(shape=(MAX_LEN,))
encoder_embedding = encoder_embedding_layer(encoder_inputs)
encoder_LSTM = LSTM(HIDDEN_UNITS, return_state=True)
encoder_LSTM_R = LSTM(HIDDEN_UNITS, return_state=True, go_backwards=True)
encoder_outputs_R, state_h_R, state_c_R = encoder_LSTM_R(encoder_embedding)
encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)

final_h = Add()([state_h, state_h_R])
final_c = Add()([state_c, state_c_R])
encoder_states = [final_h, final_c]

"""
decoder
"""
decoder_inputs = Input(shape=(MAX_LEN_S,))
decoder_embedding = decoder_embedding_layer(decoder_inputs)
decoder_LSTM = LSTM(HIDDEN_UNITS, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_LSTM(decoder_embedding, initial_state=encoder_states) 
decoder_dense = Dense(VOCAB_SIZE, activation='linear')
decoder_outputs = decoder_dense(decoder_outputs)

model= Model(inputs=[encoder_inputs,decoder_inputs], outputs=decoder_outputs)


In [29]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 400)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 400, 200)     6000000     ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 lstm (LSTM)                    [(None, 256),        467968      ['embedding[0][0]']              
                                 (None, 256),                                                 

In [30]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
filename = '/content/drive/MyDrive/Colab Notebooks/trained/model_2.sav'
pickle.dump(model, open(filename, 'wb'))

In [1]:
#num_samples = len(pad_highlight)
import numpy as np
decoder_output_data = np.zeros((700,100,30000), dtype="int32")

In [ ]:
print(pad_article.shape)
#print(decoder_output_data.shape)
print(len(pad_highlight))

In [ ]:
for i, seqs in enumerate(pad_highlight[:800]):
    for j, seq in enumerate(seqs):
        if j > 0:
            decoder_output_data[i][j][seq] = 1
          

In [ ]:
print(decoder_output_data[0])

In [ ]:
art_train, art_test, sum_train, sum_test = train_test_split(pad_article, pad_highlight, test_size=0.2)

In [ ]:
history = model.fit([pad_article,pad_highlight], 
                     target_train, 
                     epochs=EPOCHS, 
                     batch_size=BATCH_SIZE,
                     validation_data=([art_test, sum_test], target_test))